In [1]:
!pip3 install --upgrade --user --quiet google-cloud-aiplatform


In [2]:
project_id_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = project_id_output[0]
REGION = !gcloud compute project-info describe --format="value[](commonInstanceMetadata.items.google-compute-default-region)"
LOCATION = "asia-east2"

BUCKET_NAME = f"{PROJECT_ID}-model-dataset"
# BUCKET_URI = f"gs://{BUCKET_NAME}"
BUCKET_URI = "gs://peft-model-cy-aispeci-demo"

import vertexai
from vertexai.generative_models import (
    GenerativeModel,
    Part,
    HarmCategory,
    HarmBlockThreshold,
    GenerationConfig,
)
from vertexai.preview.tuning import sft
from typing import Union
import pandas as pd
from google.cloud import bigquery
from sklearn.model_selection import train_test_split
import datetime
import time
local_train_file = "hotel_train_data.jsonl"
local_validation_file = "hotel_validation_data.jsonl"
vertexai.init(project=PROJECT_ID, location=LOCATION)
print(f"✅ Vertex AI 初始化完成 - 项目: {PROJECT_ID}, 区域: {LOCATION}")


train_dataset_uri = f"gs://peft-model-cy-aispeci-demo/hotel_train_data.jsonl"
validation_dataset_uri = f"gs://peft-model-cy-aispeci-demo/hotel_validation_data.jsonl"

✅ Vertex AI 初始化完成 - 项目: cy-aispeci-demo, 区域: asia-east2


In [4]:
sft_tuning_job = sft.train(
                source_model="gemini-1.5-pro-002",
                train_dataset=train_dataset_uri,
                validation_dataset=validation_dataset_uri,
                epochs=3,
                learning_rate_multiplier=1.0,
                tuned_model_display_name="Hotel assiant-Gemini Tuning Job"
            )

Creating SupervisedTuningJob
SupervisedTuningJob created. Resource name: projects/704352985590/locations/asia-east2/tuningJobs/1130893888655982592
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/704352985590/locations/asia-east2/tuningJobs/1130893888655982592')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/asia-east2/tuning/tuningJob/1130893888655982592?project=704352985590


In [ ]:
sft_tuning_job_name = sft_tuning_job.resource_name
sft_tuning_job_name
while not sft_tuning_job.refresh().has_ended:
    time.sleep(60)
sft_tuning_job.list()
tuned_model_endpoint = sft_tuning_job.tuned_model_endpoint_name
print(tuned_model_endpoint)
tuned_model = GenerativeModel(tuned_model_endpoint)